<font color = magenta> Lixandru Andreaa-Bianca Grupa10LF382
<br> 
<font color = magenta> Pepene Adina-FLorentina Grupa10LF382

# Laborator 9

## Modele de regresie

Folositi urmatoarele seturi de date:
1. [CPU Computer Hardware](https://archive.ics.uci.edu/ml/datasets/Computer+Hardware); excludeti din dataset coloanele: vendor name, model name, estimated relative performance; se va estima coloana "published relative performance".
1. [Boston Housing](http://archive.ics.uci.edu/ml/machine-learning-databases/housing/)
1. [Wisconsin Breast Cancer](http://www.dcc.fc.up.pt/~ltorgo/Regression/DataSets.html); cautati in panelul din stanga Wisconsin Breast Cancer si urmati pasii din "My personal Notes"
1. [Communities and Crime](http://archive.ics.uci.edu/ml/datasets/communities+and+crime); stergeti primele 5 dimensiuni si trasaturile cu missing values.

Pentru fiecare set de date aplicati minim 5 modele de regresie din scikit learn. Pentru fiecare raportati: mean absolute error, mean squared error, median absolute error - a se vedea [sklearn.metrics](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) - folosind 5 fold cross validation. Valorile hiperparametrilor trebuie cautate cu grid search (cv=3)  si random search (n_iter dat de voi). Metrica folosita pentru cautarea hiperparametrilor va fi mean squared error. Raportati mediile rezultatelor atat pentru fold-urile de antrenare, cat si pentru cele de testare; indicatie: puteti folosi metoda `cross_validate` cu parametrul `return_train_score=True`, iar ca model un obiect de tip `GridSearchCV` sau `RandomizedSearchCV`.

Rezultatele vor fi trecute intr-un dataframe. Intr-o stare intermediara, valorile vor fi calculate cu semnul minus: din motive de implementare, biblioteca sklearn transforma scorurile in numere negative; a se vedea imaginea de mai jos:

![intermediate report](./images/cpu_intermediate_blurred.png)


Valorile vor fi aduse la interval pozitiv, apoi vor fi marcate cele maxime si minime; orientativ, se poate folosi imaginea de mai jos, reprezentand dataframe afisat in notebook; puteti folosi alte variante de styling pe dataframe precum la https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html#.  

Se va crea un raport final in format HTML sau PDF - fisier(e) separat(e). Raportul trebuie sa contina minimal: numele setului de date si obiectul dataframe; preferabil sa se pastreze marcajul de culori realizat in notebook.

![report](./images/cpu_results_blurred.png)

Notare:
1. Se acorda 20 de puncte din oficiu.
1. Optimizare si cuantificare de performanta a modelelor: 3 puncte pentru fiecare combinatie set de date + model = 60 de puncte
1. Documentare modele: numar modele * 2 puncte = 10 puncte. Documentati in jupyter notebook fiecare din modelele folosite, in limba romana. Puteti face o sectiune separata cu documentarea algoritmilor. Fiecare model trebuie sa aiba o descriere de minim 20 de randuri, minim o imagine asociata si minim 2 referinte bibliografice.
1. 10 puncte: export in format HTML sau PDF.



*Notare:* rezolvarea va fi incarcata pe platforma de elearning in saptamana 11-15 mai.

# Citirea si separarea datelor 

In [1]:
import pandas as pd 
import numpy as np
from typing import List
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from statistics import mean

print('Pandas version:', pd.__version__) #Pandas version: 0.23.4

Pandas version: 1.0.1


In [2]:
cpu_names = ['vendor_names', 'model', 'myct', 'mmin', 'mmax', 'cach', 'chmin', 'chmax', 'prp', 'erp']

cpu = pd.read_csv('data\machine\machine.data', names = cpu_names, header = None)

cpu = cpu.drop(columns = ['vendor_names', 'model', 'erp'])

cpu.head()

,myct,mmin,mmax,cach,chmin,chmax,prp
0,125,256,6000,256,16,128,198
1,29,8000,32000,32,8,32,269
2,29,8000,32000,32,8,32,220
3,29,8000,32000,32,8,32,172
4,29,8000,16000,32,8,16,132


In [3]:
cpu_x = cpu.values[:, :-1]
cpu_y = cpu.values[:, -1]

In [4]:
housing_names = ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis',
                           'rad', 'tax', 'ptratio', 'b', 'lstat', 'medv']

housing = pd.read_csv('data\housing\housing.data', names = housing_names, header = None, sep = r"\s+")

housing.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [5]:
housing_x = housing.values[:, :-1]
housing_y = housing.values[:, -1]

In [6]:
breast_names = ['Lymph_node', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 
                          'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave_points_mean',
                          'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se',
                          'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave_points_se',
                          'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst',
                          'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst',
                          'concave_points_worst', 'symmetry_worst', 'fractal_dimension_worst' ,'Tumor_size','Time']

breast = pd.read_csv('data/breast_cancer/r_wpbc.data', names = breast_names)

breast.head()

,Lymph_node,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,...,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst,Tumor_size,Time
0,5,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,0.07055,0.1865,...,139.70,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,31
1,2,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,0.14710,0.2419,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,61
2,0,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,0.08180,0.2333,...,159.10,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,116
3,0,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,0.10520,0.2597,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,123
4,0,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,0.10430,0.1809,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,27


In [7]:
breast_x = breast.values[:, :-1]
breast_y = breast.values[:, -1]

In [8]:
communities_names = ['state', 'county', 'community', 'communityname', 'fold', 'population', 'householdsize',
                               'racepctblack', 'racePctWhite', 'racePctAsian', 'racePctHisp', 'agePct12t21', 
                               'agePct12t29', 'agePct16t24', 'agePct65up', 'numbUrban', 'pctUrban', 'medIncome',
                               'pctWWage', 'pctWFarmSelf', 'pctWInvInc', 'pctWSocSec', 'pctWPubAsst', 'pctWRetire',
                               'medFamInc', 'perCapInc', 'whitePerCap', 'blackPerCap', 'indianPerCap', 'AsianPerCap',
                               'OtherPerCap', 'HispPerCap', 'NumUnderPov', 'PctPopUnderPov', 'PctLess9thGrade',
                               'PctNotHSGrad', 'PctBSorMore', 'PctUnemployed', 'PctEmploy', 'PctEmplManu',
                               'PctEmplProfServ', 'PctOccupManu', 'PctOccupMgmtProf', 'MalePctDivorce', 
                               'MalePctNevMarr', 'FemalePctDiv', 'TotalPctDiv', 'PersPerFam', 'PctFam2Par', 
                               'PctKids2Par', 'PctYoungKids2Par', 'PctTeen2Par', 'PctWorkMomYoungKids',
                               'PctWorkMom', 'NumIlleg', 'PctIlleg', 'NumImmig', 'PctImmigRecent', 'PctImmigRec5',
                               'PctImmigRec8', 'PctImmigRec10', 'PctRecentImmig', 'PctRecImmig5', 'PctRecImmig8',
                               'PctRecImmig10', 'PctSpeakEnglOnly', 'PctNotSpeakEnglWell', 'PctLargHouseFam',
                               'PctLargHouseOccup', 'PersPerOccupHous', 'PersPerOwnOccHous', 'PersPerRentOccHous',
                               'PctPersOwnOccup', 'PctPersDenseHous', 'PctHousLess3BR', 'MedNumBR', 'HousVacant',
                               'PctHousOccup', 'PctHousOwnOcc', 'PctVacantBoarded', 'PctVacMore6Mos', 'MedYrHousBuilt',
                               'PctHousNoPhone', 'PctWOFullPlumb', 'OwnOccLowQuart', 'OwnOccMedVal', 'OwnOccHiQuart',
                               'RentLowQ', 'RentMedian', 'RentHighQ', 'MedRent', 'MedRentPctHousInc', 
                               'MedOwnCostPctInc', 'MedOwnCostPctIncNoMtg', 'NumInShelters', 'NumStreet', 
                               'PctForeignBorn', 'PctBornSameState', 'PctSameHouse85', 'PctSameCity85', 
                               'PctSameState85', 'LemasSwornFT', 'LemasSwFTPerPop', 'LemasSwFTFieldOps', 
                               'LemasSwFTFieldPerPop', 'LemasTotalReq', 'LemasTotReqPerPop', 'PolicReqPerOffic',
                               'PolicPerPop', 'RacialMatchCommPol', 'PctPolicWhite', 'PctPolicBlack', 'PctPolicHisp',
                               'PctPolicAsian', 'PctPolicMinor', 'OfficAssgnDrugUnits', 'NumKindsDrugsSeiz',
                               'PolicAveOTWorked', 'LandArea', 'PopDens', 'PctUsePubTrans', 'PolicCars', 
                               'PolicOperBudg', 'LemasPctPolicOnPatr', 'LemasGangUnitDeploy', 'LemasPctOfficDrugUn', 
                               'PolicBudgPerPop', 'ViolentCrimesPerPop']

communities = pd.read_csv('data\communities\communities.data', names = communities_names)

communities = communities.drop(columns = ['state', 'county', 'community', 'communityname', 'fold'])
communities = communities.replace('?', np.nan)
communities = communities.dropna(axis = 1)

communities.head()

,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,...,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,ViolentCrimesPerPop
0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,...,0.12,0.42,0.50,0.51,0.64,0.12,0.26,0.20,0.32,0.20
1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,...,0.21,0.50,0.34,0.60,0.52,0.02,0.12,0.45,0.00,0.67
2,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,...,0.14,0.49,0.54,0.67,0.56,0.01,0.21,0.02,0.00,0.43
3,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,...,0.19,0.30,0.73,0.64,0.65,0.02,0.39,0.28,0.00,0.12
4,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,...,0.11,0.72,0.64,0.61,0.53,0.04,0.09,0.02,0.00,0.03


In [9]:
communities_x = communities.values[:, :-1]
communities_y = communities.values[:, -1]

# Cautarea hiperparametrilor 

In [10]:
def search_grid(model, parameters:dict, x:np.ndarray, y:np.ndarray):
    
    '''This function takes the parameters and search for best parameters.
    
    Parameters:
        model - model from sklearn 
        parameters - all the alternatives to the hyperparameters
        x, y - input and output values
    
    It returns the errors.
    '''
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 1/3)
    
    grid_search = Pipeline([('scaler', MinMaxScaler()), 
                            ('search', GridSearchCV(estimator = model, param_grid = parameters, 
                            scoring = 'neg_mean_squared_error',cv = 3, return_train_score = True))])
    grid_search.fit(x_train, y_train)
    
    return errors(grid_search, x, y)

In [11]:
def search_random(model, parameters:dict, x:np.ndarray, y:np.ndarray):
    
    '''This function takes the parameters and search for best parameters.
    
    Parameters:
        model - model from sklearn 
        parameters - all the alternatives to the hyperparameters
        x, y - input and output values
    
    It returns the errors.
    '''
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 1/3)

    rand_search = Pipeline([('scaler', MinMaxScaler()), 
                            ('search', RandomizedSearchCV(estimator = model, param_distributions = parameters, 
                            scoring = 'neg_mean_squared_error', cv = 3, return_train_score = True))])
    rand_search.fit(x_train, y_train)
    
    return errors(rand_search, x, y)

# Calcularea erorilor 

In [12]:
def errors(model, x:np.ndarray, y:np.ndarray)-> List[float]:
    
    '''Errors function calculates mean absolute error, 
    mean squared error and median absolute error for model.
    
    Parameters:
        model - model from sklearn 
        x, y - input and output values
        
    Returns a list with mean of errors:
    neg_mean_absolute_error for train and test data
    neg_mean_squared_error for train and test data
    neg_median_absolute_error for train and test data
    '''
    
    scores = cross_validate(model, x, y, cv = 5, scoring = ['neg_mean_absolute_error',
                                                            'neg_mean_squared_error',
                                                            'neg_median_absolute_error'], return_train_score = True)
    
    result = [(-1)*scores['train_neg_mean_absolute_error'].mean(), (-1)*scores['train_neg_mean_squared_error'].mean(),
              (-1)*scores['train_neg_median_absolute_error'].mean(), (-1)*scores['test_neg_mean_absolute_error'].mean(),
              (-1)*scores['test_neg_mean_squared_error'].mean(), (-1)*scores['test_neg_median_absolute_error'].mean(),
              scores['fit_time'].mean(), scores['score_time'].mean()]
    
    return result

# K-Nearest Neighbor Regression(KNN)

In [13]:
def KNN(model, parameters:dict, x:np.ndarray, y:np.ndarray):
    
    '''KNN function get params and do regression based on k-nearest
    neighbors with best parameters generated by grid search and random search.
    It generates the error values.
    
    Parameters:
        model - model from sklearn 
        parameters - all the alternatives to the hyperparameters
        x, y - input and output values
    '''
    
    main_list.append(search_grid(model, parameters, x, y))
    main_list.append(search_random(model, parameters, x, y))
    intermediar_list.append(['KNN', 'GridSearchCV'])
    intermediar_list.append(['KNN', 'RandomizedSearchCV'])

# Decision Tree Regression

In [14]:
def decision_tree(model, parameters:dict, x:np.ndarray, y:np.ndarray):
    
    '''decision_tree function get params and do regression 
    with best parameters generated by grid search and random search.
    It generates the error values.
    
    Parameters:
        model - model from sklearn 
        parameters - all the alternatives to the hyperparameters
        x, y - input and output values
    '''
    
    main_list.append(search_grid(model, parameters, x, y))
    main_list.append(search_random(model, parameters, x, y))
    intermediar_list.append(['Decision tree', 'GridSearchCV'])
    intermediar_list.append(['Decision tree', 'RandomizedSearchCV'])

# Random Forest Regression

In [15]:
def random_forest(model, parameters:dict, x:np.ndarray, y:np.ndarray):
    
    '''random_forest function get params and do regression 
    with best parameters generated by grid search and random search.
    It generates the error values.
    
    Parameters:
        model - model from sklearn 
        parameters - all the alternatives to the hyperparameters
        x, y - input and output values
    '''
    
    main_list.append(search_grid(model, parameters, x, y))
    main_list.append(search_random(model, parameters, x, y))
    intermediar_list.append(['Random Forest', 'GridSearchCV'])
    intermediar_list.append(['Random Forest', 'RandomizedSearchCV'])

# Support Vector Regression

In [16]:
def svr(model, parameters:dict, x:np.ndarray, y:np.ndarray):
    
    '''svr function get params and do regression
    with best parameters generated by grid search and random search.
    It generates the error values.
    
    Parameters:
        model - model from sklearn 
        parameters - all the alternatives to the hyperparameters
        x, y - input and output values
    '''
    
    main_list.append(search_grid(model, parameters, x, y))
    main_list.append(search_random(model, parameters, x, y))
    intermediar_list.append(['SVR', 'GridSearchCV'])
    intermediar_list.append(['SVR', 'RandomizedSearchCV'])

# Gaussian process regression

In [17]:
def gaussian_process(model, parameters:dict, x:np.ndarray, y:np.ndarray):
    
    '''gaussian_process function get params and do regression 
    with best parameters generated by grid search and random search.
    It generates the error values.
    
    Parameters:
        model - model from sklearn 
        parameters - all the alternatives to the hyperparameters
        x, y - input and output values
    '''
    
    main_list.append(search_grid(model, parameters, x, y))
    main_list.append(search_random(model, parameters, x, y))
    intermediar_list.append(['Gaussian process', 'GridSearchCV'])
    intermediar_list.append(['Gaussian process', 'RandomizedSearchCV'])

# Aplicarea modelelor pentru generarea erorilor 

In [18]:
def start_knn(x:np.ndarray, y:np.ndarray):
    
    model_knn = KNeighborsRegressor()
    parameters_knn = {'n_neighbors': [3, 4, 5, 6, 8, 9, 10, 11], 'p': [2, 3, 4, 5]}
    KNN(model_knn, parameters_knn, x, y)

In [19]:
def start_decision_tree(x:np.ndarray, y:np.ndarray):
    
    model_decision_tree = DecisionTreeRegressor()
    parameters_decision_tree = {'criterion': ['mse', 'friedman_mse'], 'max_depth': [3, 4, 5, 6, 7, 8]}
    decision_tree(model_decision_tree, parameters_decision_tree, x, y)

In [20]:
def start_random_forest(x:np.ndarray, y:np.ndarray):
    
    model_random_forest = RandomForestRegressor()
    parameters_random_forest = {'criterion': ['mse', 'friedman_mse'], 'n_estimators': [80, 100, 120], 'max_depth': [3, 4, 5, 6]}
    random_forest(model_random_forest, parameters_random_forest, x, y)

In [21]:
def start_svr(x:np.ndarray, y:np.ndarray):
    
    model_svr = SVR()
    parameters_svr = {'kernel': ['linear', 'rbf', 'poly'], 'gamma': [0.1, 1], 'C': [0.1, 1, 10, 100]}
    svr(model_svr, parameters_svr, x, y)

In [22]:
def start_gaussian_process(x:np.ndarray, y:np.ndarray):
    
    model_gaussian_process = GaussianProcessRegressor()
    parameters_gaussian_process = {'n_restarts_optimizer': [1, 5, 7, 13], 'alpha': [1e-10, 1e-4, 1e-2]}
    gaussian_process(model_gaussian_process, parameters_gaussian_process, x, y)

In [23]:
names:list = ['train_mean_absolute_error', 'train_mean_squared_error', 'train_median_absolute_error', 
              'test_mean_absolute_error', 'test_mean_squared_error', 'test_median_absolute_error', 
              'fit_time', 'score_time']
    
main_list:list = []
intermediar_list:list = []

# Colorarea valorilor maxime si minime

In [24]:
def highlight_max(s):
    '''
    highlight the maximum in a Series red.
    '''
    is_max = s == s.max()
    
    return ['background-color: red' if v else '' for v in is_max]

In [25]:
def highlight_min(s):
    '''
    highlight the minimum in a Series green.
    '''
    is_min = s == s.min()
    
    return ['background-color: green' if v else '' for v in is_min]

In [26]:
def start(x:np.ndarray, y:np.ndarray, name:str):
    
    main_list.clear()
    intermediar_list.clear()
    
    start_knn(x, y)
    start_decision_tree(x, y)
    start_random_forest(x, y)
    start_svr(x, y)
    start_gaussian_process(x, y)
    
    data_frame = pd.DataFrame(main_list, columns = names)
    intermediar_data_frame = pd.DataFrame(intermediar_list, columns = ['model_name', 'search_strategy'])
    intermediar_data_frame.style

    result_data_frame = pd.concat([intermediar_data_frame, data_frame], axis = 1)
    result_data_frame.style.apply(highlight_max, 
                               subset = pd.IndexSlice[:, names]).apply(highlight_min, 
                                                                       subset = pd.IndexSlice[:, names])
    
    result = pd.DataFrame(result_data_frame, columns=['model_name', 'search_strategy']+names)
    result = result.style.apply(highlight_max, subset= pd.IndexSlice[:, names]).apply(highlight_min,
                                                                                              subset= pd.IndexSlice[:, names])
    
    f=open(f"{name}.html","w")
    f.write(result.render())
    f.close()

In [27]:
start(cpu_x, cpu_y, "CPU Computer Hardware")

In [28]:
start(housing_x, housing_y, "Housing")

In [29]:
start(breast_x, breast_y, "Breast Cancer")

In [30]:
start(communities_x, communities_y, "Communities")

# Documentatie

## 1. K-Nearest Neighbors Regression(KNN)

K-Nearest Neighbors Regression este un algoritm simplu care stochează toate cazurile posibile și prezice o valoare pe baza unei măsuri de similaritate (de exemplu, funcții de distanță). KNN a fost utilizat în estimarea statistică și recunoașterea modelului de la începutul anilor 1970 ca tehnică non-parametrică. Regresia KNN utilizează aceleași funcții de distanță ca și clasificarea KNN. K reprezinta numarul de vecini care este cautat pentru a-l determina pe cel mai potrivit.
<br><br>Când __K = 1__ algoritmul este cunoscut ca nearest neighbor. Acesta este cel mai simplu caz. Sa presupunem că P1 este un punct pentru care trebuie sa ii prezicem eticheta. Mai întâi, găsim cel mai apropiat punct de P1 și atribuim eticheta acestuia lui P1.

<img src="resources/image1.png" width="400">

Un alt exemplu pentru __K = 3__ este:

<img src="resources/image2.png" width="400">

Urmarind plot-urile de mai jos putem observa ca folosind doar un singur vecin, fiecare punct din setul de antrenare are o influență evidentă asupra predicțiilor, iar valorile prezise trec prin toate punctele. Aceasta duce la o predicție foarte nesigură. Considerând mai mulți vecini se ajunge la predicții mai ușoare, dar acestea nu se potrivesc și cu datele de instruire.

<img src="resources/image3.png" width="1000">

Regresia K-Nearest Neighbors poate fi utilizată în cazurile în care etichetele de date sunt continue, mai degrabă decât variabile discrete. Eticheta atribuită unui punct de interogare este calculată pe baza mediilor etichetelor celor mai apropiați vecini.
În faza de clasificare, k este o constantă definită de utilizator, iar un vector nemarcat (o interogare sau un punct de testare) este clasificat prin atribuirea etichetei care este cea mai frecventă dintre eșantioanele de pregătire k cele mai apropiate de punctul de interogare.

__Metode de calcul al distantei dintre doua puncte__

Algoritmul KNN este un algoritm de invatare supervizata, care foloseste distante pentru a gasi similitudini si diferente.

__1. Distanta Euclidiana:__ este cea mai frecventă și măsoară distanța liniară dintre două probe
<br>
__2. Distanta Manhattan:__ măsoară timpul de călătorie punct-la-punct și este utilizata în mod obișnuit pentru predictorii binari

<img src="resources/image4.png" width="250">

__3. Distanta Hamming__

<img src="resources/image5.png" width="150">


Distanta Euclidiana si distanta Manhattan sunt folosite pentru variabile continue, iar distanta Hamming este folosita pentru variabile categoriale.

Cum aflam care este cel mai potrivit K?
Daca folosim un K foarte mic vom face overfitting pe setul de antrenare si nu vom avea rezultate deloc bune pentru setul de testare. Daca folosim un K foarte mare nu vom obtine rezultate bune nici pe setul de antrenare, nici pe setul de intrare. Ce facem? Solutia este curba 'elbow curve'. Un K bun poate fi selectat prin diferite tehnici euristice (ex: optimizarea hiperparametrului). 

<img src="resources/image6.png" width="550">

Un dezavantaj al clasificării de bază a „votului majoritar” apare atunci când exemple dintr-o clasă mai frecventă tind să domine predicția noului exemplu, deoarece acestea tind să fie comune printre cei mai apropiați k din cauza numărului mare. O modalitate de a depăși această problemă este ponderea clasificării, ținând cont de distanța de la punctul de testare la fiecare dintre cei mai apropiați k vecini ai săi. Clasa (sau valoarea, în problemele de regresie) din fiecare dintre cele mai apropiate k puncte este înmulțită cu o greutate proporțională cu inversul distanței de la acel punct la punctul de testare. 

Precizia algoritmului KNN poate fi sever degradată prin prezența unor caracteristici zgomotoase sau irelevante sau dacă scările de caracteristici nu sunt în concordanță cu importanța lor. 

### Bibliografie:

1. https://www.saedsayad.com/k_nearest_neighbors_reg.htm
2. https://medium.com/analytics-vidhya/k-neighbors-regression-analysis-in-python-61532d56d8e4
3. https://scikit-learn.org/stable/modules/neighbors.html#regression
4. https://www.analyticsvidhya.com/blog/2018/08/k-nearest-neighbor-introduction-regression-python/
5. https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm

## 2. Decision Tree Regression

Arborele decizional construiește modele de regresie sau clasificare sub forma unei structuri de arbori. Acesta descompune un set de date în subseturi mai mici și mai mici, în același timp se dezvoltă treptat un arbore de decizie asociat. Rezultatul final este un arbore cu noduri de decizie și noduri frunze. Un nod de decizie are două sau mai multe ramuri (de exemplu, însorit, înnorat și ploios), fiecare reprezentând valori pentru atributul testat. Nodul frunza (de exemplu, ore jucate) reprezintă o decizie cu privire la ținta numerică. Cel mai înalt nod decizie într-un arbore corespunde celui mai bun predictor numit nod rădăcină. Arborii de decizie pot gestiona atât date categorice, cât și numerice. Nodurile de decizie reprezinta conditiile, iar nodurile frunza reprezinta rezultatele.

<br>
Exemplul de mai jos care arată un arbore de decizie care determina cel mai mic numar dintre trei elemente.

<img src="resources/image8.jpg" width="650">

Regresia arborelui decizional observă trasaturile unui obiect și ataseaza în structura unui arbore trasaturile pentru a prezice date în viitor. Ieșirea continuă înseamnă că ieșirea / rezultatul nu este discret, adică nu este reprezentat doar de un set discret, cunoscut de numere sau valori.

_Exemplu de ieșire discreta:_ un model de predicție a vremii care prevede dacă va fi sau nu ploaie într-o anumită zi.
<br>_Exemplu de ieșire continuă:_ model de predicție a profitului care afirmă profitul probabil care poate fi generat din vânzarea unui produs.

Pornind de la rădăcină, datele sunt împărțite pe caracteristicile care au ca rezultat cel mai mare câștig de informații (IG- Information Gain). Într-un proces iterativ, repetăm această procedură de divizare la fiecare nod copil până când frunzele sunt pure - adică eșantioanele la fiecare nod aparțin aceleiași clase.

În practică, acest lucru poate duce la un arbore foarte adânc, cu numeroase noduri, ceea ce poate duce cu ușurință la supraîncadrare. Astfel, de obicei dorim să tăiem copacul prin stabilirea unei limite pentru adâncimea maximă a copacului.

Pentru a împărți nodurile in cele mai relevante caracteristici, trebuie să definim o funcție obiectivă pe care dorim să o optimizăm prin algoritmul de învățare. Aici, funcția noastră obiectivă este de a maximiza câștigul de informații la fiecare divizare, pe care o definim astfel:

<img src="resources/image9.png" width="500">

Pentru a utiliza un arbore de decizie pentru regresie, avem nevoie de o măsurătoare care să fie potrivită pentru variabile continue, astfel încât să definim măsura de impuritate folosind eroarea medie pătrată (MSE) a nodurilor copiilor:

<img src="resources/image10.png" width="250">

Aici, Nt este numărul de eșantioane de antrenament al nodului t, Dt este subsetul de formare al nodul t, y (i) este adevărata valoare și ŷt este valoarea țintă prevăzută:

<img src="resources/image11.png" width="125">

În practică, este important să stii cum să alegi o valoare adecvată pentru o adâncime a unui arbore pentru a face overfit sau a face undefit. Să știi cum să combini arbori de decizie pentru a forma o pădure aleatoare este, de asemenea, util, intrucat de obicei, are o performanță de generalizare mai bună decât un arbore de decizie individual, ceea ce ajută la scăderea variației modelului. De asemenea, este mai puțin sensibil la valorile din setul de date și nu necesită prea multe reglări de parametri.

### Bibliografie:

1. https://www.geeksforgeeks.org/python-decision-tree-regression-using-sklearn/
2. https://towardsdatascience.com/https-medium-com-lorrli-classification-and-regression-analysis-with-decision-trees-c43cdbc58054 -> foarte relevant
3. https://saedsayad.com/decision_tree_reg.htm

## 3. Random Forest Regression

Random Forest este o tehnică de ansamblu capabilă să îndeplinească atât sarcini de regresie, cât și de clasificare cu utilizarea mai multor arbori de decizie și o tehnică numită Bootstrap Aggregation, cunoscută de obicei ca bagging. Bagging, în metoda Random Forest, implică instruirea fiecărui arbore de decizie pe un eșantion de date diferit, unde sampling-ul se face cu înlocuire.

Arborii de decizie sunt o metodă populară pentru diferite metode de machine learning.
În special, arborii care sunt foarte adânci tind să învețe tipare extrem de neregulate: își potrivesc seturile de antrenament ai auvariație foarte mare. Random Forest este o modalitate de medie a mai multor arbori de decizie profundă, instruiți pe diferite părți ale aceluiași set de antrenament, cu scopul de a reduce variația. 

Procedura de bootstrapping duce la o performanță mai bună a modelului, deoarece scade variația modelului, fără a crește bias-urile. Aceasta înseamnă că, deși iesirile unui singur arbore sunt foarte sensibile la zgomot în setul său de formare, media mai multor arbori nu este, atâta timp cât nu exista corelații. Pur și simplu antrenarea multor arbori pe un singur set de antrenament ar oferi arbori puternic corelați (sau chiar același arbore de multe ori, dacă algoritmul de antrenare este determinist); eșantionarea bootstrap este o modalitate de a corela copacii arătându-le diferite seturi de antrenament.

<img src="resources/image12.jpg" width="400">

Metoda de mai sus descrie algoritmul de bagaj original pentru arbori. Random Forest diferă într-un singur mod de această schemă generală: folosesc un algoritm de învățare arbore modificat care selectează, la fiecare împărțire a candidatului în procesul de învățare, un subset aleator al caracteristicilor. Acest proces este uneori numit „bagaj de funcții”(feature bagging). Motivul pentru a face acest lucru este corelația arborilor dintr-un eșantion de bootstrap obișnuit: dacă una sau câteva caracteristici sunt predictori foarte puternici pentru variabila de răspuns(ieșire țintă), aceste caracteristici vor fi selectate în multe dintre arborele B, cauzându-le a deveni corelat. O analiză a modului în care bagajul și proiecția aleatorie a subspațiului contribuie la câștigurile de precizie în diferite condiții.

Diferite tipuri de modele au avantaje diferite. Modelul Random Forest este foarte bun în tratarea datelor tabulare cu caracteristici numerice sau caracteristici categorice cu mai puțin de sute de categorii. Spre deosebire de modelele liniare, Random Forest este capabil să capteze interacțiuni neliniare între caracteristici și țintă.

O notă importantă este că modelele bazate pe arbori nu sunt proiectate să funcționeze cu caracteristici foarte rare. Atunci când avem de a face cu date de intrare reduse(de exemplu, caracteristici categorice cu dimensiuni mari), putem prelucra în prealabil funcțiile rare pentru a genera statistici numerice, sau putem trece la un model liniar, care este mai potrivit pentru astfel de scenarii.

### Bibliografie:

1. https://medium.com/datadriveninvestor/random-forest-regression-9871bc9a25eb
2. https://en.wikipedia.org/wiki/Random_forest
3. https://turi.com/learn/userguide/supervised-learning/random_forest_regression.html

## 4. Epsilon-Support Vector Regression

Suportul Vector Machine poate fi folosit și ca metodă de regresie, menținând toate caracteristicile principale care caracterizează algoritmul. Deoarece rezultatul este un număr real, devine foarte dificil să prezicem informațiile disponibile, care au posibilități infinite. Însă, ideea principală este întotdeauna aceeași: minimizarea erorilor, individualizarea hiperplanului care maximizează marja. 

<img src="resources/image13.png" width="400">
<img src="resources/image14.png" width="400">

- __SVR liniar__
<img src="resources/image15.png">
- __SVR neliniar:__ Funcțiile nucleului transformă datele într-un spațiu cu caracteristici dimensionale superioare pentru a face posibilă efectuarea separației liniare.
<img src="resources/image16.png">
<img src="resources/image17.png">
Kernel functions:
<img src="resources/image27.png">

SVR ne oferă flexibilitatea de a defini cât de multă eroare este acceptabilă în modelul nostru și vom găsi o linie adecvată (sau un hiperplan cu dimensiuni mai mari) pentru a se potrivi cu datele.
<br>
Funcția obiectiv a SVR este de a minimiza coeficienții - mai precis, l norma 2 vectorului coeficient - nu eroarea pătratică. Termenul de eroare este tratat în schimb în constrângerile, unde am stabilit eroarea absolută mai mică sau egală cu o marjă specificată, numită eroare maximă, ϵ (epsilon). Putem regla epsilon pentru a obține exactitatea dorită a modelului nostru. Noile noastre funcții și constrângerile obiective sunt următoarele:
<br>
__Minimizarea:__
<img src="resources/image18a.png" width="200">
__constrângeri:__
<img src="resources/image19.png" width="200">
__Exemplu ilustrativ:__
<img src="resources/image20.png" width="400">

Pe măsură ce C crește, toleranța noastră pentru puncte în afara de ϵ crește și ea. Pe măsură ce C se apropie de 0, toleranța se apropie de 0, iar ecuația se prăbușește în cea simplificată (deși uneori imposibilă).
<br>
SVR este un algoritm puternic care ne permite să alegem cât de toleranți suntem de erori, atât printr-o marjă de eroare acceptabilă (ϵ), cât și prin reglarea toleranței noastre de a cădea în afara acelei rate de eroare acceptabile.
<br>
Deci încercăm să decidem o graniță de decizie la distanță 'e' față de planul hiper original, astfel încât punctele de date cele mai apropiate de planul hiper sau vectorii de suport să se afle în această linie de delimitare.
<img src="resources/image21.jpeg" width="400">

__De exemplu:__
<br>
Graficul de mai jos arată rezultatele unui model SVR instruit pe datele Prețurilor locuințelor din Boston. Linia roșie reprezintă linia cea mai potrivită, iar liniile negre reprezintă marja de eroare, ϵ, pe care am setat-o ​​la 5 (5.000 USD).
<img src="resources/image28.png" width="300">
Dacă setam C=1.0, graficul devine:
<img src="resources/image29.png" width="300">
Modelul de mai sus pare să se potrivească mult mai bine datelor. Putem merge cu un pas mai departe și căutăm grilă peste C pentru a obține o soluție și mai bună. Să definim o metrică de notare, în cadrul Epsilon . Această măsurătoare măsoară câte puncte din totalul setului nostru de test se încadrează în marja noastră de eroare. Putem monitoriza , de asemenea , modul în care eroarea medie absolută ( MAE ) variază în funcție de C , de asemenea.
Mai jos este prezentată o diagramă a rezultatelor căutării grilei, cu valori de C pe axa x și % în Epsilon și MAE în axele y stânga și dreapta.
<img src="resources/image31.png" width="300">
După cum putem vedea, MAE scade în general pe măsură ce C crește. Cu toate acestea, observăm un maxim în % în metrica Epsilon . Întrucât obiectivul nostru inițial al acestui model a fost maximizarea prezicerii în marja noastră de eroare (5.000 USD), dorim să găsim valoarea lui C care maximizează % în Epsilon . Astfel, C = 6.13.
Să construim un ultim model cu hyperparametrele noastre finale, ϵ = 5, C = 6.13.
<img src="resources/image30.png" width="300">

### Bibliografie:

1. https://www.saedsayad.com/support_vector_machine_reg.htm
2. https://towardsdatascience.com/an-introduction-to-support-vector-regression-svr-a3ebc1672c2
3. https://medium.com/coinmonks/support-vector-regression-or-svr-8eb3acf6d0ff

## 5. Gaussian process regression

Procesele Gaussiene (GP) sunt o metodă generică de învățare supervizată concepută pentru a rezolva problemele de regresie și clasificare probabilistică.
Avantajele proceselor gaussiene sunt:
<br>
~ Predicția interpolează observațiile
<br>
~ Predicția este probabilistică (gaussiană), astfel încât se poate calcula intervale de încredere empirică și se poate decide pe baza acelora dacă ar trebui să se adapteze predicția în anumite regiuni de interes.
<br>
~ Versatil: se pot specifica diferite nuclee.
<br>
Dezavantajele proceselor gaussiene includ:
<br>
~ Acestea nu sunt rare, adică, folosesc informații complete despre probe/caracteristici pentru a efectua predicția.
<br>
~ Se pierde eficiența în spațiile dimensionale înalte - și anume atunci când numărul de caracteristici depășește câteva zeci.
<br>

Regresia proceselor gaussiene (GPR) este o abordare nonparametrică, bayesiană a regresiei care face valuri în zona învățării automate. GPR are mai multe beneficii, funcționând bine pe seturi de date mici și având capacitatea de a furniza măsurători de incertitudine cu privire la previziuni.
<br>
Abordarea bayesiană are o distribuție a probabilității peste toate valorile posibile. 
<br>
Să presupunem o funcție liniară: __y = wx + ϵ__. Modul în care funcționează abordarea Bayesiană este specificând o distribuție anterioară, p(w) pe parametrul w și relocarea probabilităților bazate pe dovezi (adică date observate) folosind regula lui Bayes:
<img src="resources/image22.png" width="300">

Distribuția actualizată p (w | y, X) , numită distribuție posterioară , încorporează astfel informații atât din distribuția anterioară, cât și din setul de date. Pentru a obține predicții în puncte de interes nevăzute, x * , distribuția predictivă poate fi calculată prin ponderarea tuturor predicțiilor posibile prin distribuția lor posterioară calculată anterior:
<img src="resources/image23.png" width="300">
GPR calculează distribuția probabilității pe toate funcțiile admisibile care se potrivesc datelor.
<br>
Parametrii liberi, numiți hyperparametri, permit o personalizare flexibilă a Procesului Gausian pentru setul de date. Alegerea funcției de covarianță și a hiperparametrii săi se numește selecția modelului.

__De exemplu:__
<br>
Vom începe prin generarea unor date false (de la un model sinusoidal) cu bare de eroare
<img src="resources/image24.png" width="300">
Acum, vom alege o funcție de kernel (covarianță) care să modeleze aceste date, să presupunem un model mediu zero și să prezicem valorile funcției pe întreaga gamă.
<img src="resources/image25.png" width="300">
Modelul oferă un handler pentru calcularea probabilității marginalizată a datelor în conformitate cu acest model.
Deci, putem folosi acest lucru - combinat cu funcția de minimizare - pentru a se potrivi pentru parametrii de probabilitate maximă. Și rezultă modelul de probabilitate maximă.
<img src="resources/image26.png" width="300">

### Bibliografie:

1. https://towardsdatascience.com/quick-start-to-gaussian-process-regression-36d838810319
2. https://www.elen.ucl.ac.be/Proceedings/esann/esannpdf/es2013-51.pdf
3. https://scikit-learn.org/stable/modules/gaussian_process.html
4. https://george.readthedocs.io/en/latest/tutorials/first/